In [2]:
import numpy as np
from sys import path
path.append('../tools')
from tools import logsig

In [3]:
def get_expected_output(X):
    return logsig(np.dot(logsig(X * np.array([10, 10]) + np.array([-5, 5])), np.array([1, 1])) + -1)

In [4]:
vec_get_y = np.vectorize(get_expected_output)

In [5]:
x_train = np.linspace(-2, 2, 82)
y_train = vec_get_y(x_train)

In [6]:
x_train = x_train.reshape(len(x_train), 1)
y_train = y_train.reshape(len(y_train), 1)
print(x_train, y_train)

[[-2.        ]
 [-1.95061728]
 [-1.90123457]
 [-1.85185185]
 [-1.80246914]
 [-1.75308642]
 [-1.7037037 ]
 [-1.65432099]
 [-1.60493827]
 [-1.55555556]
 [-1.50617284]
 [-1.45679012]
 [-1.40740741]
 [-1.35802469]
 [-1.30864198]
 [-1.25925926]
 [-1.20987654]
 [-1.16049383]
 [-1.11111111]
 [-1.0617284 ]
 [-1.01234568]
 [-0.96296296]
 [-0.91358025]
 [-0.86419753]
 [-0.81481481]
 [-0.7654321 ]
 [-0.71604938]
 [-0.66666667]
 [-0.61728395]
 [-0.56790123]
 [-0.51851852]
 [-0.4691358 ]
 [-0.41975309]
 [-0.37037037]
 [-0.32098765]
 [-0.27160494]
 [-0.22222222]
 [-0.17283951]
 [-0.12345679]
 [-0.07407407]
 [-0.02469136]
 [ 0.02469136]
 [ 0.07407407]
 [ 0.12345679]
 [ 0.17283951]
 [ 0.22222222]
 [ 0.27160494]
 [ 0.32098765]
 [ 0.37037037]
 [ 0.41975309]
 [ 0.4691358 ]
 [ 0.51851852]
 [ 0.56790123]
 [ 0.61728395]
 [ 0.66666667]
 [ 0.71604938]
 [ 0.7654321 ]
 [ 0.81481481]
 [ 0.86419753]
 [ 0.91358025]
 [ 0.96296296]
 [ 1.01234568]
 [ 1.0617284 ]
 [ 1.11111111]
 [ 1.16049383]
 [ 1.20987654]
 [ 1.25925

In [14]:
# Create the model with keras and tensorflow
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import keras
from keras import layers

class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        
        print(data)
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compute_loss(y=y, y_pred=y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply(gradients, trainable_vars)

        # Update metrics (includes the metric that tracks the loss)
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


In [16]:
# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)


Epoch 1/3
(<tf.Tensor 'data:0' shape=(None, 32) dtype=float32>, <tf.Tensor 'data_1:0' shape=(None, 1) dtype=float32>)
(<tf.Tensor 'data:0' shape=(None, 32) dtype=float32>, <tf.Tensor 'data_1:0' shape=(None, 1) dtype=float32>)


I0000 00:00:1719674946.460850   18796 service.cc:145] XLA service 0x742a08005db0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719674946.460866   18796 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-06-29 17:29:06.469084: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-29 17:29:07.070574: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - mae: 0.3898 - loss: 0.2377
Epoch 2/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - mae: 0.3853 - loss: 0.2235
Epoch 3/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - mae: 0.3714 - loss: 0.2125


I0000 00:00:1719674947.719855   18796 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
